In [1]:
# %load Extract_MovieLens_Data.py


# # This notebook is to extract data from Movie Lens
# * The data contents are explained in http://files.grouplens.org/papers/ml-1m-README.txt
# 
# ## users.dat
# 
# UserID::Gender::Age::Occupation::Zip-code
# - Gender is denoted by a "M" for male and "F" for female
# - Age is chosen from the following ranges:
# 
# 	*  1:  "Under 18"
# 	* 18:  "18-24"
# 	* 25:  "25-34"
# 	* 35:  "35-44"
# 	* 45:  "45-49"
# 	* 50:  "50-55"
# 	* 56:  "56+"
# 
# - Occupation is chosen from the following choices:
# 
# 	*  0:  "other" or not specified
# 	*  1:  "academic/educator"
# 	*  2:  "artist"
# 	*  3:  "clerical/admin"
# 	*  4:  "college/grad student"
# 	*  5:  "customer service"
# 	*  6:  "doctor/health care"
# 	*  7:  "executive/managerial"
# 	*  8:  "farmer"
# 	*  9:  "homemaker"
# 	* 10:  "K-12 student"
# 	* 11:  "lawyer"
# 	* 12:  "programmer"
# 	* 13:  "retired"
# 	* 14:  "sales/marketing"
# 	* 15:  "scientist"
# 	* 16:  "self-employed"
# 	* 17:  "technician/engineer"
# 	* 18:  "tradesman/craftsman"
# 	* 19:  "unemployed"
# 	* 20:  "writer"
# 
# ## movies.dat
# MovieID::Title::Genres
# 
# ## ratings.dat
# UserID::MovieID::Rating::Timestamp

# In[1]:

import pandas as pd

# In[3]:

users_df = pd.read_csv("./ml-1m/users.dat"
                    , sep='::'
                    , header=None
                    , names=["UserID", "Gender", "Age", "Occupation", "Zip-code"])
ocupation_codes = {'ocupation_code': [x for x in range(21)]
                   , 'Occupation_name': ["other or not specified", "academic/educator", "artist"
                                  , "clerical/admin", "college/grad student", "customer service"
                                  , "doctor/health care", "executive/managerial", "farmer"
                                  , "homemaker", "K-12 student", "lawyer", "programmer", "retired"
                                  , "sales/marketing" ,"scientist", "self-employed", "technician/engineer"
                                  , "tradesman/craftsman", "unemployed", "writer"]
                  }
ocupation_codes = pd.DataFrame(ocupation_codes)
users_df = users_df.merge(ocupation_codes, left_on=["Occupation"], right_on=["ocupation_code"], how='left')
users_df = users_df.drop(["Occupation", "ocupation_code"], axis=1).rename(columns={'Occupation_name': 'Occupation'})


# In[4]:

movies_org_df = pd.read_csv("./ml-1m/movies.dat"
                            , sep='::'
                            , header=None
                            , names=["MovieID", "Title", "Genres"])
rows = []
for _, row in movies_org_df.iterrows():
    for gen in row.Genres.split('|'):
        rows.append([row['MovieID'], row['Title'], gen])
movies_df = pd.DataFrame(rows, columns=movies_org_df.columns)


# In[5]:

ratings_df = pd.read_csv("./ml-1m/ratings.dat"
                         , sep='::'
                         , header=None
                         , names=["UserID", "MovieID", "Rating", "Timestamp"])
ratings_df['rating_dt'] = pd.to_datetime(ratings_df['Timestamp'],unit='s')

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:60: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:79: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:92: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [2]:
print(users_df.columns)
print(movies_df.columns)
print(ratings_df.columns)

Index(['UserID', 'Gender', 'Age', 'Zip-code', 'Occupation'], dtype='object')
Index(['MovieID', 'Title', 'Genres'], dtype='object')
Index(['UserID', 'MovieID', 'Rating', 'Timestamp', 'rating_dt'], dtype='object')


In [3]:
df = ratings_df.merge(users_df, on=['UserID'], how='left').merge(movies_df, on=['MovieID'], how='left')

In [4]:
df.columns

Index(['UserID', 'MovieID', 'Rating', 'Timestamp', 'rating_dt', 'Gender',
       'Age', 'Zip-code', 'Occupation', 'Title', 'Genres'],
      dtype='object')

In [5]:
import datetime as dt
import numpy as np

In [6]:
ratings_2001_Q3 = df[(df.rating_dt < dt.datetime(2000, 10, 1)) & (df.rating_dt >= dt.datetime(2000, 7, 1))]

In [7]:
ratings_2001_Q3[["Rating_mean", "Age_mean"]] = ratings_2001_Q3.groupby(["Genres", "Gender", "Occupation"])[['Rating', 'Age']] \
.transform(np.mean).rename(columns={
    "Rating": "Rating_mean"
    , "Age": "Age_mean"
})

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:2440: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [8]:
a = ratings_2001_Q3.groupby(["Genres", "Gender", "Occupation"])[['Rating', 'Age']].agg([np.mean, np.std])
a = a.reset_index()
# a.columns = a.columns.get_level_values(0)
a.columns.values
a.columns = [x[:-1] if x.endswith('_') else x for x in ['_'.join(x) for x in a.columns.values]]
a

,Genres,Gender,Occupation,Rating_mean,Rating_std,Age_mean,Age_std
0,Action,F,K-12 student,3.480769,1.491789,1.000000,0.000000
1,Action,F,academic/educator,3.350866,1.106950,33.192431,11.456293
2,Action,F,artist,3.383436,1.190168,30.049080,11.641762
3,Action,F,clerical/admin,3.722307,0.995569,31.418816,9.880010
4,Action,F,college/grad student,3.470425,1.141231,22.175601,5.954492
5,Action,F,customer service,3.374690,1.252283,32.168734,7.214987
6,Action,F,doctor/health care,3.553776,1.057131,35.269260,9.747967
7,Action,F,executive/managerial,3.794118,1.049907,32.926471,9.324766
8,Action,F,farmer,3.427907,1.141061,51.339535,2.504402
9,Action,F,homemaker,3.530120,1.109723,32.706827,7.427070


In [9]:
#df["Rating_mean"] = df.groupby(["Occupation", "Genres"])['Rating'].transform(np.mean)
#df.sort_values(by="Rating_mean", ascending=False).groupby(["Occupation"]).head(5)
# df.groupby(["Occupation", "Genres"])['Rating'].agg(np.mean).reset_index().rename(columns={"Rating": "Rating_avg"}).sort_values(by="Rating_avg", ascending=False).groupby(["Occupation"]).head(5).sort_values(by=["Occupation", "Rating_avg"])
df.groupby(["Occupation", "Genres"])['Rating'].agg(np.mean).reset_index().rename(columns={"Rating": "Rating_avg"}).sort_values(by="Rating_avg", ascending=False).groupby(["Occupation"]).head(5).sort_values(by=["Occupation", "Rating_avg"])

,Occupation,Genres,Rating_avg
12,K-12 student,Mystery,3.636612
5,K-12 student,Crime,3.687085
7,K-12 student,Drama,3.782167
16,K-12 student,War,3.880144
9,K-12 student,Film-Noir,4.212766
29,academic/educator,Musical,3.701586
25,academic/educator,Drama,3.754293
34,academic/educator,War,3.882950
24,academic/educator,Documentary,3.984887
27,academic/educator,Film-Noir,4.082613


In [10]:
# Find top 5 ratings for each occupation
#df = df.groupby(['Occupation', 'Genres']).agg({'Rating': np.mean}).reset_index()
df.assign(Rating_avg=df.groupby(['Occupation', 'Genres'])[['Rating']].transform(np.mean))[['Occupation', 'Genres', 'Rating_avg']] \
.drop_duplicates() \
.sort_values(by="Rating_avg", ascending=False) \
.groupby(["Occupation"]).head(5).sort_values(by=["Occupation", "Rating_avg"])

,Occupation,Genres,Rating_avg
4960,K-12 student,Mystery,3.636612
91,K-12 student,Crime,3.687085
0,K-12 student,Drama,3.782167
46,K-12 student,War,3.880144
4953,K-12 student,Film-Noir,4.212766
1747,academic/educator,Musical,3.701586
1120,academic/educator,Drama,3.754293
1145,academic/educator,War,3.882950
2130,academic/educator,Documentary,3.984887
1708,academic/educator,Film-Noir,4.082613


In [11]:
# df.groupby(["Occupation", "Genres"]).transform(lambda x : x if print(type(x)) else x)
# df.groupby(["Occupation", "Genres"])['Rating'].transform(lambda x : x if print(len(x)) else x)

In [16]:
# df.groupby(["Occupation", "Genres"])['Rating'].transform(lambda x: x.mean())
# df.groupby(["Occupation", "Genres"]).transform(lambda x: x.mean())
# df.groupby(["Occupation", "Genres"])['Rating'].apply(lambda x: x.mean())
df.groupby(["Occupation", "Genres"])['Rating'].agg(lambda x: x.mean())

Occupation         Genres     
K-12 student       Action         3.497116
                   Adventure      3.425658
                   Animation      3.463956
                   Children's     3.220679
                   Comedy         3.497200
                   Crime          3.687085
                   Documentary    3.581633
                   Drama          3.782167
                   Fantasy        3.298039
                   Film-Noir      4.212766
                   Horror         3.237795
                   Musical        3.556738
                   Mystery        3.636612
                   Romance        3.624415
                   Sci-Fi         3.443795
                   Thriller       3.554131
                   War            3.880144
                   Western        3.513333
academic/educator  Action         3.392063
                   Adventure      3.424278
                   Animation      3.693399
                   Children's     3.459286
                   Come

In [13]:
df.groupby(["Occupation", "Genres"]).transform(lambda x: x if print(len(x)) else x)

6067
6067
6067
6067
6067
6067
6067
6067
6067
6067
6067
3726
2164
3888
9465
1355
98
6000
1275
235
1905
1410
732
2990
3932
4212
1385
300
18673
10215
3242
5600
29949
6326
794
34769
2763
1852
5079
3847
3730
14433
11795
14723
6006
1715
11277
5986
2163
3375
17982
3957
553
19273
1760
1000
3676
2387
2026
7824
7110
8683
3072
973
7078
3700
1321
2226
11870
2469
285
11533
1043
677
2210
1518
1356
5210
4022
5922
1950
659
35152
17910
7036
11264
48672
10772
970
43546
5225
1770
10383
5442
4740
18913
20352
25302
8205
2091
6493
3228
974
1597
7908
1734
97
6369
934
329
2390
888
718
2649
3789
4238
1348
546
8601
4446
1565
2596
13148
2800
302
14332
1216
705
2576
1646
1469
5971
5146
6781
2572
717
28863
14138
3451
6251
35784
8738
707
38888
3664
1865
7479
3881
4270
15453
16316
20654
7981
2695
868
528
134
261
939
198
10
835
152
41
182
100
82
373
550
497
164
51
2098
1299
693
1281
5065
699
35
4152
409
130
542
694
391
2681
1065
1710
664
191
4793
2563
726
1209
7526
1795
191
7832
610
537
1249
874
920
2989
2776
3632
15

,UserID,MovieID,Rating,Timestamp,rating_dt,Gender,Age,Zip-code,Title
0,1,1193,5,978300760,2000-12-31 22:12:40,F,1,48067,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,978302109,2000-12-31 22:35:09,F,1,48067,James and the Giant Peach (1996)
2,1,661,3,978302109,2000-12-31 22:35:09,F,1,48067,James and the Giant Peach (1996)
3,1,661,3,978302109,2000-12-31 22:35:09,F,1,48067,James and the Giant Peach (1996)
4,1,914,3,978301968,2000-12-31 22:32:48,F,1,48067,My Fair Lady (1964)
5,1,914,3,978301968,2000-12-31 22:32:48,F,1,48067,My Fair Lady (1964)
6,1,3408,4,978300275,2000-12-31 22:04:35,F,1,48067,Erin Brockovich (2000)
7,1,2355,5,978824291,2001-01-06 23:38:11,F,1,48067,"Bug's Life, A (1998)"
8,1,2355,5,978824291,2001-01-06 23:38:11,F,1,48067,"Bug's Life, A (1998)"
9,1,2355,5,978824291,2001-01-06 23:38:11,F,1,48067,"Bug's Life, A (1998)"


In [14]:
df.assign(max_ratings = ratings_df.groupby("MovieID")['Rating'].transform(np.max))

,UserID,MovieID,Rating,Timestamp,rating_dt,Gender,Age,Zip-code,Occupation,Title,Genres,max_ratings
0,1,1193,5,978300760,2000-12-31 22:12:40,F,1,48067,K-12 student,One Flew Over the Cuckoo's Nest (1975),Drama,5.0
1,1,661,3,978302109,2000-12-31 22:35:09,F,1,48067,K-12 student,James and the Giant Peach (1996),Animation,5.0
2,1,661,3,978302109,2000-12-31 22:35:09,F,1,48067,K-12 student,James and the Giant Peach (1996),Children's,5.0
3,1,661,3,978302109,2000-12-31 22:35:09,F,1,48067,K-12 student,James and the Giant Peach (1996),Musical,5.0
4,1,914,3,978301968,2000-12-31 22:32:48,F,1,48067,K-12 student,My Fair Lady (1964),Musical,5.0
5,1,914,3,978301968,2000-12-31 22:32:48,F,1,48067,K-12 student,My Fair Lady (1964),Romance,5.0
6,1,3408,4,978300275,2000-12-31 22:04:35,F,1,48067,K-12 student,Erin Brockovich (2000),Drama,5.0
7,1,2355,5,978824291,2001-01-06 23:38:11,F,1,48067,K-12 student,"Bug's Life, A (1998)",Animation,5.0
8,1,2355,5,978824291,2001-01-06 23:38:11,F,1,48067,K-12 student,"Bug's Life, A (1998)",Children's,5.0
9,1,2355,5,978824291,2001-01-06 23:38:11,F,1,48067,K-12 student,"Bug's Life, A (1998)",Comedy,5.0
